In [11]:
import xarray as xr
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import holoviews as hv
import geoviews as gv
import geoviews.feature as gf
from cartopy import crs

from scipy import stats
import datetime as dt

hv.notebook_extension()
%matplotlib inline

# Open MOD13A1 NetCDFs, Rename vars

In [12]:
ds_all=xr.open_mfdataset(
    '/g/data/oe9/project/team-drip/NDVI_EVI_MODIS_MDB/NDVI_stack/MOD13A1_NDVI_EVI_????.nc',
    chunks=dict(time=12, x=200,y=200)) 

ds_all=ds_all.rename({'__xarray_dataarray_variable__': 'ndvi_evi','y': 'lat', 'x': 'lon'})
ds_all['band'] = ('band', ['dayofyear', 'ndvi', 'evi'])
ds_all

<xarray.Dataset>
Dimensions:   (band: 3, lat: 3147, lon: 3346, time: 388)
Coordinates:
  * band      (band) <U9 'dayofyear' 'ndvi' 'evi'
  * lat       (lat) float64 -24.59 -24.59 -24.6 -24.6 -24.6 -24.61 -24.61 ...
  * lon       (lon) float64 138.6 138.6 138.6 138.6 138.6 138.6 138.6 138.6 ...
  * time      (time) datetime64[ns] 2000-02-18 2000-03-05 2000-03-21 ...
Data variables:
    ndvi_evi  (time, band, lat, lon) int16 dask.array<shape=(388, 3, 3147, 3346), chunksize=(12, 3, 200, 200)>

In [13]:
ndvi=ds_all.ndvi_evi.sel(band='ndvi')
ndvi

<xarray.DataArray 'ndvi_evi' (time: 388, lat: 3147, lon: 3346)>
dask.array<shape=(388, 3147, 3346), dtype=int16, chunksize=(12, 200, 200)>
Coordinates:
    band     <U9 'ndvi'
  * lat      (lat) float64 -24.59 -24.59 -24.6 -24.6 -24.6 -24.61 -24.61 ...
  * lon      (lon) float64 138.6 138.6 138.6 138.6 138.6 138.6 138.6 138.6 ...
  * time     (time) datetime64[ns] 2000-02-18 2000-03-05 2000-03-21 ...
Attributes:
    crs:        +init=epsg:4326
    res:        [0.00416201 0.00416201]
    is_tiled:   1
    transform:  [ 4.16200895e-03  0.00000000e+00  1.38565764e+02  0.00000000e...

In [17]:
# def nanneg(data):
#     return data(data<0)==np.nan

# xr.apply_ufunc(nanneg,ndvi)
ndvi[ndvi<0,:,:]=np.nan
#  y[np.array([0,2,4]), 1]



IndexError: Unlabeled multi-dimensional array cannot be used for indexing: [[[False False False ... False False False]
  [False False False ... False False False]
  [False False False ... False False False]
  ...
  [False False False ... False False False]
  [False False False ... False False False]
  [False False False ... False False False]]

 [[False False False ... False False False]
  [False False False ... False False False]
  [False False False ... False False False]
  ...
  [False False False ... False False False]
  [False False False ... False False False]
  [False False False ... False False False]]

 [[False False False ... False False False]
  [False False False ... False False False]
  [False False False ... False False False]
  ...
  [False False False ... False False False]
  [False False False ... False False False]
  [False False False ... False False False]]

 ...

 [[False False False ... False False False]
  [False False False ... False False False]
  [False False 

In [46]:
ndvi=ds_all.ndvi_evi.sel(band='ndvi')
ndvi

<xarray.DataArray 'ndvi_evi' (time: 388, lat: 3147, lon: 3346)>
dask.array<shape=(388, 3147, 3346), dtype=int16, chunksize=(20, 3147, 3346)>
Coordinates:
    band     <U9 'ndvi'
  * lat      (lat) float64 -24.59 -24.59 -24.6 -24.6 -24.6 -24.61 -24.61 ...
  * lon      (lon) float64 138.6 138.6 138.6 138.6 138.6 138.6 138.6 138.6 ...
  * time     (time) datetime64[ns] 2000-02-18 2000-03-05 2000-03-21 ...
Attributes:
    crs:        +init=epsg:4326
    res:        [0.00416201 0.00416201]
    is_tiled:   1
    transform:  [ 4.16200895e-03  0.00000000e+00  1.38565764e+02  0.00000000e...

In [ ]:
import numpy as np
import xarray as xr
import bottleneck

def covariance_gufunc(x, y):
    return ((x - x.mean(axis=-1, keepdims=True))
            * (y - y.mean(axis=-1, keepdims=True))).mean(axis=-1)

def pearson_correlation_gufunc(x, y):
    return covariance_gufunc(x, y) / (x.std(axis=-1) * y.std(axis=-1))

def spearman_correlation_gufunc(x, y):
    x_ranks = bottleneck.rankdata(x, axis=-1)
    y_ranks = bottleneck.rankdata(y, axis=-1)
    return pearson_correlation_gufunc(x_ranks, y_ranks)

def spearman_correlation(x, y, dim):
    return xr.apply_ufunc(
        spearman_correlation_gufunc, x, y,
        input_core_dims=[[dim], [dim]],
        dask='parallelized',
        output_dtypes=[float])